#随机森林算法梳理

## 1、集成学习的概念
> 它本身不是一个单独的机器学习算法，而是通过构建并结合多个机器学习器来完成学习任务。也就是我们常说的“博采众长”。集成学习可以用于分类问题集成，回归问题集成，特征选取集成，异常点检测集成等等，可以说所有的机器学习领域都可以看到集成学习的身影。

集成学习有两个主要的问题需要解决，
  - 第一是如何得到若干个个体学习器[弱学习器]，
  - 第二是如何选择一种结合策略，将这些个体学习器[弱学习器]集合成一个强学习器。

## 2、个体学习器的概念
- 个体学习器是相对于集成学习来说的，作为单个学习器，它通常是由一个现有的学习算法从训练数据产生，如C4.5决策树算法、BP神经网络算法等。个体学习器代表的是单个学习器，集成学习代表的是多个学习器的结合。

    - 若集成中只包含同种类型的个体学习器，例如“决策树集成”中全是决策树，这样的集成是“同质”的。同质集成中的个体学习器亦称“基学习器”，相应的学习算法称为基学习算法。
    - 若集成中包含不同类型的个体学习器，例如同时包含决策树和神经网络，这样的集成是“异质”的。这时，个体学习器一般不称为基学习器，常称为“组件学习器”或直接称为个体学习器。

## 3、boosting bagging的概念、异同点
- 3.1 boosting
    - Boosting是一族可将弱学习器提升为强学习器的算法。它的基本原理：先从初始训练集训练出一个基学习器，再根据基学习器的表现对训练样本分布进行调整，**[提高被错误分类的样本的权重]**，降低被正确分类的样本的权重，使得先前基学习器做错的训练样本在后续受到更多的关注，然后基于调整后的样本分布来训练下一个基学习器；如此重复进行，直至基学习器数目达到事先指定的值T，最后将这T个基学习器进行加权结合。
![boosting](https://raw.githubusercontent.com/xmj-datawhale/adv-algorithm/master/img/ada-bosting.png)
    - Boosting族最著名的代表是AdaBoost，但标准的AdaBoost只可用于处理二分类任务。其算法流程如下：
    ![boosting](https://raw.githubusercontent.com/xmj-datawhale/adv-algorithm/master/img/adv-boosting-algorithm.jpg)
    - 从偏差-方差分解的角度看，Boosting主要关注降低偏差，因此Boosting基于泛化性能相当弱的学习器构建出很强的集成。
- 3.2 bagging
    - Bagging是并行式集成学习方法最著名的代表。它直接基于自助采样法(bootstrap sampling)。给定包含m个样本的数据集，我们先随机取出一个样本放入采样集中，再把该样本放回初始数据集，使得下次采样时该样本仍有可能被选中，这样，经过m轮随机采样，我们得到m个样本的采样集，初始训练集中有的样本在采样集中多次出现，有的则从未出现，约63.2%的样本出现在采样集中，而未出现的约36.8%的样本可用作验证集来对后续的泛化性能进行“包外估计”。
    - 照这样，我们可以采样出T个含m个训练样本的采样集，然后基于每个采样集训练出一个基学习器，然后将这些基学习器进行结合。在对预测输出进行结合时，Bagging通常对分类任务使用简单投票法，对回归任务使用简单平均法，这就是Bagging的基本流程。
    ![boosting](https://raw.githubusercontent.com/xmj-datawhale/adv-algorithm/master/img/adv-bagging.jpg)

    - 从偏差-方差分解的角度看，Bagging主要关注降低方差，因此它在不剪枝的决策树、神经网络等易受到样本扰动的学习器上效用更明显。
    - 随机森林是Bagging的一个扩展变体。在样本扰动的基础上，加上属性扰动。
- 3.3.Bagging与Boosting比较
>Bagging是个体学习器不存在强依赖关系，可以同时生成的并行化方法；Boosting是个体学习器间存在强依赖关系、必须串行生成的序列化方法。
从偏差-方差分解角度看，Bagging主要关注降低方差，而Boosting主要关注降低偏差。

## 4、理解不同的结合策略(平均法，投票法，学习法)
- 1.平均法：
    ①简单平均法
    ②加权平均法

    - 当个体学习器性能相差较大时宜用加权平均法，而在个体学习器性能相近时宜用简单平均法。
- 2.投票法

    ①绝对多数投票法：哪类得票过半数，则预测为该类，否则拒绝预测。
    
    ②相对多数投票法：哪类得票最多，若同时多个类得票最多，则随机从中选取一个。
    
    ③加权投票法

- 3.学习法

    - 当训练数据很多时，一种更为强大的结合策略是使用“学习法”，即通过另一个学习器来进行结合。Stacking是学习法的典型代表。我们把个体学习器称为初级学习器，用于结合的学习器称为次级学习器或元学习器。

> Stack先从初始数据集训练出初级学习器，然后“生成”一个新数据集用于训练次级学习器。在这个新数据集中，初级学习器的输出被当作样例输入特征，而初始样本的标记仍被当作样例标记。
## 5、随机森林的思想
> 随机森林是Bagging的一个扩展变体，在理解了Bagging方法后，随机森林学习起来就容易多了。RF在以决策树作为基学习器构建Bagging集成的基础上，进一步在决策树的训练过程中加入了随机属性的选择。具体来说，传统决策树在选择划分属性时是在当前结点的所有候选属性（假定有d个）中选择一个最优属性；而在RF中，对基决策树的每个结点，先从该结点的候选属性集合中随机选择一个包含k个属性的子集，然后再从这个子集中选择一个最优属性用于划分。抽取的属性数k的选择比较重要，一般推荐 [公式] 。由此，随机森林的基学习器的“多样性”不仅来自样本的扰动，还来自属性的扰动，使得最终集成的泛化能力进一步增强。

- 随机森林特点主要是：

    - 个体学习器为决策树
    - 对训练样本进行采样
    - 对属性进行随机采样

## 6、随机森林的推广
- 1.extra trees

    extra trees是RF的一个变种, 原理几乎和RF一模一样，仅有区别有：
    
    ①对于每个决策树的训练集，RF采用的是随机采样bootstrap来选择采样集作为每个决策树的训练集，而extra trees一般不采用随机采样，即每个决策树采用原始训练集。
    
    ②在选定了划分特征后，RF的决策树会基于基尼系数，均方差之类的原则，选择一个最优的特征值划分点，这和传统的决策树相同。但是extra trees比较的激进，他会随机的选择一个特征值来划分决策树。
    
    从第二点可以看出，由于随机选择了特征值的划分点位，而不是最优点位，这样会导致生成的决策树的规模一般会大于RF所生成的决策树。也就是说，模型的方差相对于RF进一步减少，但是偏倚相对于RF进一步增大。在某些时候，extra trees的泛化能力比RF更好。
- 2.Totally Random Trees Embedding
## 7、随机森林的优缺点
- 随机森林的优点：

    ①训练可以高度并行化，可以有效运行在大数据集上。
    
    ②由于对决策树候选划分属性的采样，这样在样本特征维度较高的时候，仍然可以高效的训练模型。
    
    ③由于有了样本和属性的采样，最终训练出来的模型泛化能力强。
    
    ④可以输出各特征对预测目标的重要性。
    
    ⑤对部分特征的缺失容忍度高。
    
    ⑥袋外数据可用作验证集来检验模型的有效性，不用额外划分数据集。
    
    随机森林的缺点：
    
    ①在某些噪声比较大的样本集上，随机森林容易陷入过拟合。
    
    ②取值划分比较多的特征容易对随机森林的决策产生更大的影响，从而影响拟合的模型效果
## 8、随机森林在sklearn中的参数解释

随机森林的分类学习器为RandomForestClassifier，回归学习器为RandomForestRegressor.

    RandomForestClassifier(
      n_estimators=10, criterion=’gini’,
      max_depth=None,min_samples_split=2,
      min_samples_leaf=1, min_weight_fraction_leaf=0.0,
      max_features=’auto’,max_leaf_nodes=None,
      min_impurity_decrease=0.0, min_impurity_split=None, 
      bootstrap=True, oob_score=False, n_jobs=None, 
      random_state=None, verbose=0, warm_start=False, class_weight=None)
参数解释：

控制bagging框架的参数

estimators：随机森林中树的棵树，即要生成多少个基学习器（决策树）。
boostrap：是否采用自助式采样生成采样集。
obb_score：是否使用袋外数据来估计模型的有效性。
控制决策树的参数

criterion：选择最优划分属性的准则，默认是"gini"，可选"entropy"。
max_depth：决策树的最大深度
max_features：随机抽取的候选划分属性集的最大特征数（属性采样）
min_samples_split：内部节点再划分所需最小样本数。默认是2，可设置为整数或浮点型小数。
min_samples_leaf：叶子节点最少样本数。默认是1，可设置为整数或浮点型小数。
max_leaf_nodes：最大叶子结点数。默认是不限制。
min_weight_fraction_leaf：叶子节点最小的样本权重和。默认是0。
min_impurity_split：节点划分最小不纯度。
其他参数：

n_jobs：并行job的个数
verbose：是否显示任务进程
可调用方法：

predict_proba：计算预测的概率值
predict(x)：预测
predict_log_proba(x)：计算出预测的对数概率值
可调用的属性：

estimators_：列出决策树参数
feature_importances_：列出变量重要性
n_features：
n_outputs_：
obb_score_：袋外数据测试效果
obb_prediction_：袋外数据预测结果
## 9、随机森林的应用场景

参考

[随机森林算法梳理（Random Forest）](https://zhuanlan.zhihu.com/p/57965634)
